<a href="https://colab.research.google.com/github/dbrody112/Frequentist-Machine-Learning/blob/main/Dan_Brody_Market_Basket_Analysis_Last_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#loading in data


from google.colab import files
uploaded = files.upload()

Saving newestbestcolumns.csv to newestbestcolumns.csv


In [ ]:
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot
import seaborn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline,Pipeline
import xgboost as xgb
from sklearn.metrics import accuracy_score
import io
import plotly.express as px
from numba import cuda
from numba import jit

df_values = pd.read_csv(io.BytesIO(uploaded['newestbestcolumns.csv']))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#loading in keys for data

from google.colab import files
uploaded = files.upload()

Saving newestinfoforbestcolumns.csv to newestinfoforbestcolumns.csv


In [ ]:

df_keys = pd.read_csv(io.BytesIO(uploaded['newestinfoforbestcolumns.csv']))

In [ ]:
#@title
#functions

from numpy import *
from itertools import combinations
def get_support(df,minSupport):
    pp = []
    for cnum in range(1, len(df.columns)+1):
        for cols in combinations(df, cnum):
            s = df[list(cols)].all(axis=1).sum()
            pp.append([",".join(cols), s])
    sdf = pd.DataFrame(pp, columns=["Pattern", "Support"])
    return sdf[sdf["Support"] >= minSupport]

#creates a dataset

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

#finds unique keys in dataSet

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict    

#returns possible sets from C1 that occur more frequently than the minSupport

def scanD(D, Ck, minSupport):

  #checks for subsets

    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    print(ssCnt)
    supportData = {}
    for key in ssCnt:

#computes frequency for each key and outputs the keys with frequency greater than minSupport

        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData


#returns possible pairs from Lk 

def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

#combines all functions aforementioned together to return possible pairs with frequency greater than minSupport

def apriori(dataSet, minSupport = 0.5):
    (C1) = list(createC1(dataSet))
    (D) = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

#generating pairs with confidence greater than minConf

def generateRules(L, supportData, minConf=0.7,minLift = 0.7):  
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf,minLift)
            else:
                calcConfandLift(freqSet, H1, supportData, bigRuleList, minConf,minLift)
    return bigRuleList         

#calculating confidence that result is correct and prints out results

def calcConfandLift(freqSet, H, supportData, brl, minConf=0.7,minLift = 0.7):
    prunedH = [] 
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        lift = conf/(supportData[freqSet]*supportData[conseq])
        if conf >= minConf and lift >= minLift: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf,'lift:',lift)
            brl.append((freqSet-conseq, conseq, conf,lift))
            prunedH.append(conseq)
    return prunedH


#sets rules such as there needs to be at least two sets to merge   

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7,minLift = 0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConfandLift(freqSet, Hmp1, supportData, brl, minConf)
        
        #need at least two sets to merge
        
        if (len(Hmp1) > 1):    
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [ ]:
#GENERATING POSSIBLE PAIRS WITH FREQ GREATER THAN MINSUPPORT

L1_real,suppData2_real = apriori(df_values.values,0.65)

{frozenset({0}): 1, frozenset({3}): 26, frozenset({9}): 91, frozenset({14}): 36, frozenset({21}): 26, frozenset({26}): 21, frozenset({34}): 44, frozenset({1236}): 34, frozenset({3537}): 188, frozenset({3539}): 119, frozenset({3545}): 30, frozenset({3550}): 27, frozenset({3555}): 116, frozenset({3559}): 74, frozenset({3563}): 65, frozenset({3567}): 115, frozenset({3570}): 119, frozenset({3572}): 70, frozenset({3578}): 71, frozenset({3608}): 54, frozenset({3614}): 31, frozenset({3636}): 11, frozenset({4101}): 38, frozenset({4107}): 120, frozenset({4111}): 54, frozenset({4116}): 60, frozenset({4120}): 34, frozenset({4124}): 25, frozenset({4136}): 55, frozenset({4141}): 31, frozenset({4146}): 50, frozenset({4150}): 41, frozenset({4155}): 48, frozenset({4160}): 46, frozenset({4164}): 11, frozenset({4170}): 50, frozenset({4175}): 52, frozenset({4178}): 59, frozenset({4186}): 3, frozenset({4193}): 32, frozenset({4200}): 46, frozenset({4206}): 46, frozenset({4211}): 48, frozenset({4217}): 53, 

In [ ]:
suppData2_real

{frozenset({0}): 0.005208333333333333,
 frozenset({3}): 0.13541666666666666,
 frozenset({9}): 0.4739583333333333,
 frozenset({14}): 0.1875,
 frozenset({21}): 0.13541666666666666,
 frozenset({26}): 0.109375,
 frozenset({34}): 0.22916666666666666,
 frozenset({1236}): 0.17708333333333334,
 frozenset({3537}): 0.9791666666666666,
 frozenset({3539}): 0.6197916666666666,
 frozenset({3545}): 0.15625,
 frozenset({3550}): 0.140625,
 frozenset({3555}): 0.6041666666666666,
 frozenset({3559}): 0.3854166666666667,
 frozenset({3563}): 0.3385416666666667,
 frozenset({3567}): 0.5989583333333334,
 frozenset({3570}): 0.6197916666666666,
 frozenset({3572}): 0.3645833333333333,
 frozenset({3578}): 0.3697916666666667,
 frozenset({3608}): 0.28125,
 frozenset({3614}): 0.16145833333333334,
 frozenset({3636}): 0.057291666666666664,
 frozenset({4101}): 0.19791666666666666,
 frozenset({4107}): 0.625,
 frozenset({4111}): 0.28125,
 frozenset({4116}): 0.3125,
 frozenset({4120}): 0.17708333333333334,
 frozenset({4124

In [ ]:
#GENERATING PAIRS WITH CONFIDENCE GREATER THAN MINCONF

rules = generateRules(L1_real,suppData2_real,minConf = 0.91,minLift = 1.0)

Streaming output truncated to the last 5000 lines.
frozenset({4123}) --> frozenset({4177, 4300}) conf: 1.0 lift: 2.3219954648526078
frozenset({4177}) --> frozenset({4123, 4300}) conf: 1.0 lift: 2.3219954648526078
frozenset({4220}) --> frozenset({4177, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4177}) --> frozenset({4220, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4215}) --> frozenset({4177, 4261}) conf: 1.0 lift: 2.3405714285714287
frozenset({4177}) --> frozenset({4261, 4215}) conf: 0.992063492063492 lift: 2.3405714285714287
frozenset({4209}) --> frozenset({4177, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4177}) --> frozenset({4209, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4180}) --> frozenset({4177, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4177}) --> frozenset({4180, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4148}) --> frozenset({4177, 4261}) conf: 1.0 lift: 2.3219954648526078
frozenset({4177}) --> frozenset({4148, 4261}

KeyError: ignored

In [ ]:
#PUTTING DATA IN DATAFRAME

df = pd.DataFrame(rules,columns = ['set1','set2','confidence','lift'])

In [ ]:
#CONVERTING SET1 AND SET2 TO MORE EASILY ANALYZE

set1list = [list(x) for x in df.values[:,0]]
set2list = [list(x) for x in df.values[:,1]]

In [ ]:
set1list_one = []
set1list_two = []
set1list_three = []
set1list_four = []
set2list_one = []
set2list_two = []
set2list_three = []
set2list_four = []

In [ ]:
#DIVIDING SET1 AND SET2 INTO 8 COLUMNS (ONE FOR EACH NUMBER IN SET)

for i in set1list:
  if(len(i) == 4):
    set1list_one.append(i[0])
    set1list_two.append(i[1])
    set1list_three.append(i[2])
    set1list_four.append(i[3])
  elif(len(i) == 3):
    set1list_one.append(i[0])
    set1list_two.append(i[1])
    set1list_three.append(i[2])
    set1list_four.append(0)
  elif(len(i)==2):
    set1list_one.append(i[0])
    set1list_two.append(i[1])
    set1list_three.append(0)
    set1list_four.append(0)
  else:
    set1list_one.append(i[0])
    set1list_two.append(0)
    set1list_three.append(0)
    set1list_four.append(0)




In [ ]:
for i in set2list:
  if(len(i) == 4):
    set2list_one.append(i[0])
    set2list_two.append(i[1])
    set2list_three.append(i[2])
    set2list_four.append(i[3])
  elif(len(i) == 3):
    set2list_one.append(i[0])
    set2list_two.append(i[1])
    set2list_three.append(i[2])
    set2list_four.append(0)
  elif(len(i)==2):
    set2list_one.append(i[0])
    set2list_two.append(i[1])
    set2list_three.append(0)
    set2list_four.append(0)
  else:
    set2list_one.append(i[0])
    set2list_two.append(0)
    set2list_three.append(0)
    set2list_four.append(0)

In [ ]:
df['set1list_one'] = set1list_one
df['set1list_two'] = set1list_two
df['set1list_three'] = set1list_three
df['set1list_four'] = set1list_four
df['set2list_one'] = set2list_one
df['set2list_two'] = set2list_two
df['set2list_three'] = set2list_three
df['set2list_four'] = set2list_four

In [ ]:
df.head()

,set1,set2,confidence,lift,set1list_one,set1list_two,set1list_three,set1list_four,set2list_one,set2list_two,set2list_three,set2list_four
0,(2),(3537),0.981595,1.202976,2,0,0,0,3537,0,0,0
1,(2),(4261),1.000000,1.177914,2,0,0,0,4261,0,0,0
2,(2),(4300),1.000000,1.177914,2,0,0,0,4300,0,0,0
3,(2),(4491),0.944785,1.270559,2,0,0,0,4491,0,0,0
4,(19),(4261),1.000000,1.238710,19,0,0,0,4261,0,0,0


In [ ]:
len(df)

891

In [ ]:
#OBTAINING UNIQUE KEYS FROM THE SETS

unique_numbers = []
for i in ['set1list_one', 'set1list_two',
       'set1list_three', 'set1list_four', 'set2list_one', 'set2list_two',
       'set2list_three', 'set2list_four']:
       print(f"the unique keys for {i} are {df[i].unique()}")
       for j in df[i].unique():
         if(j not in unique_numbers):
           unique_numbers.append(j)

the unique keys for set1list_one are [   2   19 4261 3537 4262 4300 4491 4495 4499 6925]
the unique keys for set1list_two are [   0 4300 4261    2 4491 4495 4262 4499 6925]
the unique keys for set1list_three are [   0 4495 6925 4261 4300 4262 4499 4491]
the unique keys for set1list_four are [   0 4495 6925 4261 4499]
the unique keys for set2list_one are [3537 4261 4300 4491 4495 4499 6925]
the unique keys for set2list_two are [   0 4261 4300 6925 4491 4495 4499]
the unique keys for set2list_three are [   0 4261 4300 4495 6925 4491 4499]
the unique keys for set2list_four are [   0 6925 4495 4300 4491]


In [ ]:
unique_numbers

[2, 19, 4261, 3537, 4262, 4300, 4491, 4495, 4499, 6925, 0]

In [ ]:
#DEFINING THE UNIQUE KEYS

new_dict = {
    4276: 'did not know that low tar and light cigarettes are just as harmful as regular cigarettes',
    2: 'has looked for information about cancer from a source',
    19: 'did not answer if they were a cancer caregiver',
    4482: 'family member had/has cancer',
    4258: 'if a cigarette was advertised as “low nicotine” the user thinks it would be as addictive as any other cigarette',
    4269: 'has seen statements that cigarettes are designed to enhance the delivery of nicotine',
    3537:'does not know average calories per day',
    4261:'has seen tobacco ad',
    4262:'has seen statements about the health of smoking',
    4300:'has had cancer',
    4491:'thinks unexplained bleeding could be a cause of cancer',
    4495: 'does think that a change in bowel or bladder habits could be a cause for cancer',
    4499: 'does think that unexpected weight loss could a cause for cancer',
    6925: 'either chose not to answer or drinks a max average of 1.2 drinks per week',
    0:""
}




In [ ]:
#APPENDING NEW COLUMNS TO THE DATASET THAT LINKS DEFINITIONS TO THEIR KEY


new_columns = ['1st word','second word','third word','fourth word', '--> first word ','--> second word','--> third word','-->fourth word']
b = 0
for i in ['set1list_one', 'set1list_two',
       'set1list_three', 'set1list_four', 'set2list_one', 'set2list_two',
       'set2list_three', 'set2list_four']:
       if(i != 'set1list_one'):
         df[new_columns[b-1]] = appending
         print(b)
       b = b + 1
       appending = []
       for i in df[i]:
         if(i in new_dict):
           appending.append(new_dict[i])
         else:
           appending.append(0)



1
2
3
4
5
6
7


In [ ]:
x = []      
for i in df['set2list_four']:
  if(i in new_dict):
    x.append(new_dict[i])
  else:
    x.append(0)
df['--> fourth word'] = x


In [ ]:
df.columns

Index(['set1', 'set2', 'confidence', 'lift', 'set1list_one', 'set1list_two',
       'set1list_three', 'set1list_four', 'set2list_one', 'set2list_two',
       'set2list_three', 'set2list_four', '1st word', 'second word',
       'third word', 'fourth word', '--> first word ', '--> second word',
       '--> third word', '--> fourth word'],
      dtype='object')

In [ ]:
#SORTING VALUES BY CONFIDENCE

df = df.sort_values(by='confidence',ascending = True)

In [ ]:
#PRINTING OUT AND STORING SERIES OF EVENTS

chances = []
for i in range(len(df['1st word'])):
  chances.append([i,(f"if {df['1st word'][i]} [and] {df['second word'][i]} [and]  {df['third word'][i]}  [and]  {df['fourth word'][i]} [[then]] \n  {df['--> first word '][i]}  [and]   {df['--> second word'][i]}  [and]   {df['--> third word'][i]} [and]   {df['--> fourth word'][i]} /n with confidence of {df['confidence'][i]} and lift of {df['lift'][i]}"),df['confidence'][i],df['lift'][i]])
  print((f"if {df['1st word'][i]} [and] {df['second word'][i]} [and]  {df['third word'][i]}  [and]  {df['fourth word'][i]} [[then]] \n  {df['--> first word '][i]}  [and]   {df['--> second word'][i]}  [and]   {df['--> third word'][i]} [and]   {df['--> fourth word'][i]} /n with confidence of {df['confidence'][i]}"))
  print('\n')


if has looked for information about cancer from a source [and]  [and]    [and]   [[then]] 
  does not know average calories per day  [and]     [and]    [and]    /n with confidence of 0.9815950920245399


if has looked for information about cancer from a source [and]  [and]    [and]   [[then]] 
  has seen tobacco ad  [and]     [and]    [and]    /n with confidence of 1.0


if has looked for information about cancer from a source [and]  [and]    [and]   [[then]] 
  has had cancer  [and]     [and]    [and]    /n with confidence of 1.0


if has looked for information about cancer from a source [and]  [and]    [and]   [[then]] 
  thinks unexplained bleeding could be a cause of cancer  [and]     [and]    [and]    /n with confidence of 0.9447852760736196


if did not answer if they were a cancer caregiver [and]  [and]    [and]   [[then]] 
  has seen tobacco ad  [and]     [and]    [and]    /n with confidence of 1.0


if did not answer if they were a cancer caregiver [and]  [and]    [and]   [[th

In [ ]:
report= pd.DataFrame(chances,columns = ['#','event','confidence','lift'])

In [ ]:
report.values[0]

array([0,
       'if has looked for information about cancer from a source [and]  [and]    [and]   [[then]] \n  does not know average calories per day  [and]     [and]    [and]    /n with confidence of 0.9815950920245399 and lift of 1.2029761127790106',
       0.9815950920245399, 1.2029761127790106], dtype=object)